# Using Meeshkan for developing Kaggle kernels
For a full example of how to, for example, submit kernels from the command-line, see the example page in [GitHub](https://github.com/Meeshkan/meeshkan-client/tree/kaggle-kernel-example/examples/kaggle).

### Install dependencies

In [ ]:
!pip install keras tensorflow meeshkan pandas scikit-learn

In [ ]:
import os
import random

import meeshkan

import keras

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.gaussian_process import GaussianProcessRegressor

Check that data has been downloaded and it's available:

In [ ]:
print(os.listdir("../input"))

### Setting up Meeshkan

To use Meeshkan, you need to 
1. register at [meeshkan.com](https://meeshkan.com) and get our API key
1. setup Slack integration as instructed in the [docs](https://www.meeshkan.com/docs)
1. initialize the credentials and start the agent.

The second step can be achieved with the `meeshkan.init(token=YOUR_API_KEY)` command. **Make sure to replace the example API key below with your own key!**

In [ ]:
meeshkan.init(token="YOUR_API_KEY")  # REPLACE THIS WITH YOUR API_KEY

Meeshkan agent can be used to schedule multiple machine learning jobs, but as we're just running stuff sequentially, we can simply use the [as_blocking_job](https://meeshkan-client.readthedocs.io/en/latest/#meeshkan.as_blocking_job) decorator to group runs as jobs. Execute the cell below to ensure you get Slack notifications.

In [ ]:
import time
@meeshkan.as_blocking_job(job_name="test-job", report_interval_secs=10)
def train():
    for i in range(10):
        meeshkan.report_scalar("Random number", random.uniform(0, 1))
        time.sleep(2)
        
train()

If everything works as expected, we can get rocking with machine learning!

### Run your job

To keep this demo as simple as possible, we will only use the `Age` column to try and predict `AdoptionSpeed`. First load data and peek into first rows:

In [ ]:
train_df = pd.read_csv("../input/train/train.csv")
train_df.head()

Check what the columns look like:

In [ ]:
train_df.info()

Let us check how `Age` compares to `AdoptionSpeed`:

In [ ]:
train_df.plot(x="Age", y="AdoptionSpeed", kind="scatter")

Obviously the correlation is not very good and age of zero should be taken into account.

Let us create a training and validation set from `train_df`:

In [ ]:
from sklearn.model_selection import train_test_split

TRAIN_COLUMNS = ['Age']

X = train_df[TRAIN_COLUMNS].values
y = train_df['AdoptionSpeed'].values

# Normalize input and output
X_scaler = sklearn.preprocessing.MinMaxScaler()
y_scaler = sklearn.preprocessing.MinMaxScaler()
X = X_scaler.fit_transform(X)
y = y_scaler.fit_transform(y.reshape(-1, 1))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True)

In [ ]:
X

In [ ]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)

In [ ]:
plt.plot(X_train, y_train, 'bo')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import LambdaCallback
from keras.optimizers import SGD

def build_model():
    model = Sequential()
    model.add(Dense(10, input_dim=len(TRAIN_COLUMNS)))
    model.add(Activation('relu'))
    model.add(Dense(5))
    model.add(Activation('relu'))
    model.add(Dense(1, input_dim=len(TRAIN_COLUMNS)))

    model.compile(loss='mse', optimizer=SGD(lr=1e-3), metrics=['mae'])
    return model

def on_epoch_end(epoch, logs):
    try:
        meeshkan.report_scalar("Train loss", float(logs['loss']), "Train mae", float(logs['mean_absolute_error']))
    except Exception as e:
        print(e)

EPOCHS = 100
VALIDATION_INTERVAL = 1

@meeshkan.as_blocking_job(job_name="train-keras", report_interval_secs=60)
def train(model):
    print('Training -----------')
    
    import time
    time.sleep(1)
    # Add a condition to send notification when MAE is small enough
    meeshkan.add_condition("Train mae", condition=lambda value: value < 0.3)
    
    # Report scalars to Meeshkan at every epoch end
    meeshkan_callback = LambdaCallback(on_epoch_end=on_epoch_end)
    history = model.fit(X_train,
                        y_train,
                        epochs=EPOCHS,
                        batch_size=256,
                        callbacks = [meeshkan_callback])
    return history

model = build_model()

train(model)

You should have received notifications for job being started and, finally, for the job being finished. Let's plot the predictions for fun.

In [ ]:
y_pred = model.predict(X_val)

# Perform inverse scalings
X_val_orig = X_scaler.inverse_transform(X_val)

plt.plot(X_val_orig, y_scaler.inverse_transform(y_pred), 'bo', label='Prediction')
plt.plot(X_val_orig, y_scaler.inverse_transform(y_val), 'rs', label='Adoption speed')
plt.legend()

### Teardown

Once we're done, we can stop the Meeshkan agent cleanly:

In [ ]:
meeshkan.stop()